<a href="https://colab.research.google.com/github/dicky-alfansyah/automatics_check_cookies/blob/main/check_cookies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rarfile

In [ ]:
import os
import zipfile
import rarfile
import requests
import pandas as pd
from google.colab import files

def check_cookies_valid_netflix(cookies):
    url = "https://www.netflix.com/browse"
    response = requests.get(url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def check_cookies_valid_spotify(cookies):
    url = "https://www.spotify.com/in-en/account/overview/"
    response = requests.get(url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def check_cookies_valid_crunchyroll(cookies):
    url = "https://www.crunchyroll.com/account/preferences"
    response = requests.get(url, cookies=cookies, allow_redirects=False)
    return "Active" if response.status_code == 200 else "Expired"

def read_cookies(file):
    cookies = {}
    with open(file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 7:
                name = parts[5]
                value = parts[6]
                cookies[name] = value
    return cookies

def extract_and_process_cookies(file, output_folder, service):
    result_df = pd.DataFrame(columns=['File Name', 'Cookies Status'])

    if file.endswith(('.zip', '.rar')):
        with (zipfile.ZipFile(file, 'r') if file.endswith('.zip') else rarfile.RarFile(file, 'r')) as archive:
            for file_info in archive.infolist():
                if file_info.filename.endswith('.txt'):
                    file_name = os.path.basename(file_info.filename)
                    file_path = os.path.join(output_folder, file_name)
                    with open(file_path, 'wb') as f:
                        f.write(archive.read(file_info.filename))
                    cookies = read_cookies(file_path)
                    if service == 'Netflix':
                        cookies_status = check_cookies_valid_netflix(cookies)
                    elif service == 'Spotify':
                        cookies_status = check_cookies_valid_spotify(cookies)
                    elif service == 'Crunchyroll':
                        cookies_status = check_cookies_valid_crunchyroll(cookies)
                    temp_df = pd.DataFrame({'File Name': [file_name], 'Cookies Status': [cookies_status]})
                    result_df = pd.concat([result_df, temp_df], ignore_index=True)

    return result_df

if not os.path.exists('data'):
    os.makedirs('data')

while True:

    print("\033[1mSCRIPT by. M DICKY ALFANSYAH\033[0m")
    print("\nMenu:")
    print("1. Check Netflix cookies")
    print("2. Check Spotify cookies")
    print("3. Check Crunchyroll cookies")
    print("4. Exit\n")

    choice = input("Enter your choice: ")

    if choice in ('1', '2', '3'):
        if choice == '1':
            service = 'Netflix'
        elif choice == '2':
            service = 'Spotify'
        elif choice == '3':
            service = 'Crunchyroll'
        print(f"Please upload a ZIP or RAR file for {service} cookies:")

        uploaded_files = files.upload()
        file_names = list(uploaded_files.keys())
        file_name = file_names[0] if file_names else None

        if file_name:
            content = uploaded_files[file_name]

            with open(file_name, 'wb') as f:
                f.write(content)

            result_df = extract_and_process_cookies(file_name, 'data', service)

            os.remove(file_name)

            active_cookies_df = result_df[result_df['Cookies Status'] == 'Active']

            if not active_cookies_df.empty:
                active_cookies_df = active_cookies_df.sort_values(by='Cookies Status')
                print(active_cookies_df)

                # Save to Excel
                excel_file_name = f"Status_Cookies_{service}_Active.xlsx"
                active_cookies_df.to_excel(excel_file_name, index=False)
                print(f"Saved {service} active cookies status to {excel_file_name}\n")
            else:
                print("No active cookies found.\n")

        else:
            print("No file uploaded.\n")

    elif choice == '4':
        print("Thank you for using this script ❤︎❤︎❤︎.")
        break
    else:
        print("Invalid choice. Please enter a valid choice.\n")
